In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import pickle

In [3]:
# Load data and model
df_bfs_data = pd.read_csv('bfs_municipality_and_tax_data.csv', sep=',', encoding='utf-8')
df_bfs_data['tax_income'] = df_bfs_data['tax_income'].str.replace("'", "").astype(float)

with open('random_forest_regression.pkl', 'rb') as f:
    random_forest_model = pickle.load(f)

In [4]:
locations = {
    "Zürich": 261,
    "Kloten": 62,
    "Uster": 198,
    "Illnau-Effretikon": 296,
    "Feuerthalen": 27,
    "Pfäffikon": 177,
    "Ottenbach": 11,
    "Dübendorf": 191,
    "Richterswil": 138,
    "Maur": 195,
    "Embrach": 56,
    "Bülach": 53,
    "Winterthur": 230,
    "Oetwil am See": 157,
    "Russikon": 178,
    "Obfelden": 10,
    "Wald (ZH)": 120,
    "Niederweningen": 91,
    "Dällikon": 84,
    "Buchs (ZH)": 83,
    "Rüti (ZH)": 118,
    "Hittnau": 173,
    "Bassersdorf": 52,
    "Glattfelden": 58,
    "Opfikon": 66,
    "Hinwil": 117,
    "Regensberg": 95,
    "Langnau am Albis": 136,
    "Dietikon": 243,
    "Erlenbach (ZH)": 151,
    "Kappel am Albis": 6,
    "Stäfa": 158,
    "Zell (ZH)": 231,
    "Turbenthal": 228,
    "Oberglatt": 92,
    "Winkel": 72,
    "Volketswil": 199,
    "Kilchberg (ZH)": 135,
    "Wetzikon (ZH)": 121,
    "Zumikon": 160,
    "Weisslingen": 180,
    "Elsau": 219,
    "Hettlingen": 221,
    "Rüschlikon": 139,
    "Stallikon": 13,
    "Dielsdorf": 86,
    "Wallisellen": 69,
    "Dietlikon": 54,
    "Meilen": 156,
    "Wangen-Brüttisellen": 200,
    "Flaach": 28,
    "Regensdorf": 96,
    "Niederhasli": 90,
    "Bauma": 297,
    "Aesch (ZH)": 241,
    "Schlieren": 247,
    "Dürnten": 113,
    "Unterengstringen": 249,
    "Gossau (ZH)": 115,
    "Oberengstringen": 245,
    "Schleinikon": 98,
    "Aeugst am Albis": 1,
    "Rheinau": 38,
    "Höri": 60,
    "Rickenbach (ZH)": 225,
    "Rafz": 67,
    "Adliswil": 131,
    "Zollikon": 161,
    "Urdorf": 250,
    "Hombrechtikon": 153,
    "Birmensdorf (ZH)": 242,
    "Fehraltorf": 172,
    "Weiach": 102,
    "Männedorf": 155,
    "Küsnacht (ZH)": 154,
    "Hausen am Albis": 4,
    "Hochfelden": 59,
    "Fällanden": 193,
    "Greifensee": 194,
    "Mönchaltorf": 196,
    "Dägerlen": 214,
    "Thalheim an der Thur": 39,
    "Uetikon am See": 159,
    "Seuzach": 227,
    "Uitikon": 248,
    "Affoltern am Albis": 2,
    "Geroldswil": 244,
    "Niederglatt": 89,
    "Thalwil": 141,
    "Rorbas": 68,
    "Pfungen": 224,
    "Weiningen (ZH)": 251,
    "Bubikon": 112,
    "Neftenbach": 223,
    "Mettmenstetten": 9,
    "Otelfingen": 94,
    "Flurlingen": 29,
    "Stadel": 100,
    "Grüningen": 116,
    "Henggart": 31,
    "Dachsen": 25,
    "Bonstetten": 3,
    "Bachenbülach": 51,
    "Horgen": 295
}

In [5]:
def predict_apartment(rooms, area, town):
    bfs_number = locations.get(town)
    if bfs_number is None:
        return "Town not found. Please select a valid town."
    df = df_bfs_data[df_bfs_data['bfs_number'] == bfs_number].copy()
    df.reset_index(inplace=True)
    df.loc[0, 'rooms'] = rooms
    df.loc[0, 'area'] = area
    prediction = random_forest_model.predict(df[['rooms', 'area', 'pop', 'pop_dens', 'frg_pct', 'emp', 'tax_income']])
    return np.round(prediction[0], 0)

In [6]:
def compare_apartments(rooms1, area1, town1, rooms2, area2, town2):
    price1 = predict_apartment(rooms1, area1, town1)
    price2 = predict_apartment(rooms2, area2, town2)
    difference = np.abs(price1 - price2)
    return price1, price2, difference

In [7]:
# Gradio Blocks Interface with structured layout
demo = gr.Blocks()

with demo:
    gr.Markdown("## 🏘️ Apartment Price Comparison")
    gr.Markdown("Enter details for two apartments to see individual prices and the difference.")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Apartment 1")
            rooms1 = gr.Number(label="Rooms")
            area1 = gr.Number(label="Area (m²)")
            town1 = gr.Dropdown(choices=list(locations.keys()), label="Town")
        with gr.Column():
            gr.Markdown("### Apartment 2")
            rooms2 = gr.Number(label="Rooms")
            area2 = gr.Number(label="Area (m²)")
            town2 = gr.Dropdown(choices=list(locations.keys()), label="Town")

    compare_btn = gr.Button("Compare Prices")

    with gr.Row():
        price1 = gr.Number(label="Price Apartment 1")
        price2 = gr.Number(label="Price Apartment 2")
        difference = gr.Number(label="Price Difference")

    compare_btn.click(
        fn=compare_apartments,
        inputs=[rooms1, area1, town1, rooms2, area2, town2],
        outputs=[price1, price2, difference]
    )

    gr.Examples(
        examples=[
            [4, 100, "Zürich", 3.5, 80, "Winterthur"],
            [3, 75, "Uster", 2.5, 60, "Dietlikon"]
        ],
        inputs=[rooms1, area1, town1, rooms2, area2, town2]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
